<a href="https://colab.research.google.com/github/richa11101982/-git-clone-https-github.com-android-codelab-android-kmp/blob/main/Burger's_Equation_Physics_Informed_Neural_Network_(PINN)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Great\! Let's proceed with setting up the **PyTorch** structure for your **Burger's Equation Physics-Informed Neural Network (PINN)**.

PyTorch is excellent for PINNs because its **dynamic computation graph** makes calculating the required second-order derivatives via automatic differentiation very straightforward.

-----

## 💻 Step 1: Setting up the Environment

You'll need PyTorch and NumPy.

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import grad

-----

## 🧠 Step 2: Define the Neural Network ($u_{NN}$)

This network takes $(x, t)$ as input and outputs $u$.

In [2]:
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        # Sequential model definition
        self.model = nn.Sequential()

        # Build the layers
        for i in range(len(layers) - 2):
            self.model.add_module(f"layer_{i}", nn.Linear(layers[i], layers[i+1]))
            self.model.add_module(f"activation_{i}", nn.Tanh())

        # Output layer (no activation for the final output u)
        self.model.add_module("output", nn.Linear(layers[-2], layers[-1]))

    def forward(self, x, t):
        # Concatenate x and t as the input vector
        return self.model(torch.cat([x, t], dim=1))

# Example Configuration: 2 inputs (x, t), 4 hidden layers of 20 neurons, 1 output (u)
# layers = [2, 20, 20, 20, 20, 1]
# model = PINN(layers)

-----

## 🧪 Step 3: Define the Physics Residual ($f$)

This is the core function where automatic differentiation is performed to calculate the derivatives needed for the Burger's equation residual:

$$f(x, t) = \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} - \nu \frac{\partial^2 u}{\partial x^2}$$

In [3]:
def physics_residual(model, x_physics, t_physics, nu):
    # Ensure inputs require gradient tracking
    x = x_physics.clone().requires_grad_(True)
    t = t_physics.clone().requires_grad_(True)

    # 1. Network Output (u)
    u = model(x, t)

    # --- 2. Calculate First Derivatives (u_t and u_x) ---
    # Create a tensor of ones for the vector-Jacobian product (vjp)
    # The output 'u' is a column vector of solutions for all points
    u_t_x = grad(u, (t, x), grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)
    u_t = u_t_x[0] # du/dt
    u_x = u_t_x[1] # du/dx

    # --- 3. Calculate Second Derivative (u_xx) ---
    # The second derivative is found by taking the gradient of u_x with respect to x
    u_xx = grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]

    # --- 4. Calculate the Residual (f) ---
    # f = u_t + u * u_x - nu * u_xx
    f = u_t + u * u_x - nu * u_xx

    # We want f to be zero, so the loss is the MSE of f
    return f

-----

## 📊 Step 4: Data Sampling

We need three sets of points for the three loss terms ($\mathcal{L}_{IC}$, $\mathcal{L}_{BC}$, $\mathcal{L}_{Physics}$).

In [4]:
# Parameters
nu = 0.01 / np.pi # Viscosity term
N_IC = 50        # Initial Condition points
N_BC = 50        # Boundary Condition points
N_PHYSICS = 1000 # Collocation/Physics points (most critical)

# Convert all data to PyTorch Tensors
def create_data_tensors():
    # 1. Initial Condition (IC) Points: t=0, x in [-1, 1]
    x_ic = torch.linspace(-1, 1, N_IC).view(-1, 1)
    t_ic = torch.zeros_like(x_ic)
    # True value for IC: u(x, 0) = -sin(pi * x)
    u_ic_true = -torch.sin(np.pi * x_ic)

    # 2. Boundary Condition (BC) Points: x=-1 and x=1, t in [0, 1]
    t_bc = torch.rand(N_BC).view(-1, 1)
    x_bc_neg1 = -torch.ones_like(t_bc) # x = -1
    x_bc_pos1 = torch.ones_like(t_bc)  # x = 1
    # True value for BC: u(-1, t) = 0 and u(1, t) = 0
    u_bc_true = torch.zeros_like(t_bc)

    # Concatenate boundary points
    x_bc = torch.cat([x_bc_neg1, x_bc_pos1], dim=0)
    t_bc = torch.cat([t_bc, t_bc], dim=0)


    # 3. Physics (Collocation) Points: t in [0, 1], x in [-1, 1]
    # Sample randomly in the entire spatio-temporal domain
    x_physics = 2 * torch.rand(N_PHYSICS).view(-1, 1) - 1 # x in [-1, 1]
    t_physics = torch.rand(N_PHYSICS).view(-1, 1)         # t in [0, 1]

    return x_ic, t_ic, u_ic_true, x_bc, t_bc, u_bc_true, x_physics, t_physics

# --- The full training loop would now assemble these components ---

The next logical step is to combine these components into the **training loop**, where the losses are calculated and the optimizer updates the network weights.

Would you like to proceed with the **training loop setup** and optimization strategy?